## Purpose: Try different models-- Part6.
### forest

In [1]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../Resources/clean_data.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,St. Louis Cardinals,2019,1033,114,43,936,8313.0,3,2771,3847,...,0.319,456,4,895,33,3896,56,1.29,21,0
1,Arizona Diamondbacks,2019,1010,83,45,945,8538.0,2,2846,3901,...,0.315,472,7,925,24,4001,53,1.28,35,0
2,Kansas City Royals,2019,990,105,45,954,8421.0,6,2807,3842,...,0.346,543,5,816,24,4125,39,1.46,34,0
3,Houston Astros,2019,875,54,50,954,8589.0,6,2863,3788,...,0.284,432,7,1074,27,3929,67,1.14,31,0
4,Tampa Bay Rays,2019,975,92,53,963,8760.0,11,2920,3948,...,0.291,409,6,1037,26,3985,59,1.16,40,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
120,San Francisco Giants,2015,1639,136,72,1458,13143.0,6,4381,6092,...,0.303,631,11,1309,43,6048,87,1.21,40,0
121,Washington Nationals,2015,1425,142,73,1458,13137.0,17,4379,5877,...,0.300,612,12,1476,46,6036,95,1.19,47,0
122,Houston Astros,2015,1599,135,77,1458,13212.0,18,4404,6080,...,0.314,701,8,1396,44,6180,84,1.29,98,0
123,Detroit Tigers,2015,1537,148,75,1449,12852.0,5,4284,5896,...,0.320,721,8,1232,47,6048,86,1.32,44,0
124,Boston Red Sox,2015,1427,139,75,1458,12957.0,37,4319,5821,...,0.314,694,5,1362,43,6073,93,1.27,52,0


In [4]:
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(2344,)
(2344, 46)
['A', 'DP', 'E', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'H', 'HBP', 'HR', 'NP_x', 'OBP', 'OPS_x', 'R', 'RBI', 'SAC', 'SB', 'SLG', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'TBF', 'W', 'WHIP', 'WP']


#### STEP2: Split data. Train and test the model.

In [5]:
def tree(features, target):
    '''
    INPUT: 
    -features = dataset without team names and year.
    -target = labels.
    
    OUTPUT:
    -model = model output
    
    DESCRIPTION:
    -dataset is taken in and split into test and train sets.
    -model is fit
    -model is tested
    '''
    
    # split data.
    X_train, X_test, y_train, y_test = train_test_split(features, target)

    # train model.
    model = RandomForestClassifier(n_jobs=-1,
                                   n_estimators=100)
    model.fit(X_train, y_train)

    # predict.
    prediction = model.predict(X_test)

    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    return model

In [6]:
model = tree(features, target)

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       557
           1       0.00      0.00      0.00        29

   micro avg       0.95      0.95      0.95       586
   macro avg       0.48      0.50      0.49       586
weighted avg       0.90      0.95      0.93       586



/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


This is terrible.  Let's try upsampling.

#### STEP3: Upsampling

In [7]:
# upsample for a more balanced dataset.
def upsample(dataset, no_samples):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    -n_samples = number of minority_unsampled.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -dataset is taken in and split into minority and majority classes.
    -dataset is then upsampled for the mainority class
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate majority and minority classes.
    df_majority = dataset.loc[dataset["winners"] == 0]
    df_minority = dataset.loc[dataset["winners"] == 1]

    # upsample minority class.
    df_minority_unsampled = resample(df_minority,
                                    replace=True,
                                    n_samples=no_samples,
                                    random_state=123)

    # combine majority class with upsampled minority class.
    df_upsampled = pd.concat([df_majority, df_minority_unsampled])

    # separate features and target.
    y = df_upsampled["winners"]
    X = df_upsampled[feature_columns]
    
    model = tree(X, y)
    return model

In [8]:
model_100 = upsample(team_data_new, 2234)
print ("_______________________________")
model_50 = upsample(team_data_new, 1117)
print ("_______________________________")
model_25 = upsample(team_data_new, 559)
print ("_______________________________")
model_12 = upsample(team_data_new, 250)
print ("_______________________________")
model_6 = upsample(team_data_new, 200)
print ("_______________________________")
model_3 = upsample(team_data_new, 150)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00       547
           1       0.99      1.00      1.00       570

   micro avg       1.00      1.00      1.00      1117
   macro avg       1.00      1.00      1.00      1117
weighted avg       1.00      1.00      1.00      1117

_______________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       566
           1       1.00      1.00      1.00       272

   micro avg       1.00      1.00      1.00       838
   macro avg       1.00      1.00      1.00       838
weighted avg       1.00      1.00      1.00       838

_______________________________
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       564
           1       0.95      0.99      0.97       135

   micro avg       0.99      0.99      0.99       699
   macro avg       0.97      0.99      0.98       699
weighted a

Probably overfit.

In [9]:
def predict_the_winner(model, year, team_data):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -model = the saved model.
    -team_data = complete dataframe with all data.
    -year = the year want to look at.
    
    OUTPUT:
    -printed prediction.
    
    DESCRIPTION:
    -data from year of interest is isolated.
    -the data are scaled.
    -the prediction is made.
    -print out the resulting probability and the name of the team.
    '''
    
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()

    # set features (no team, year, winners).
    # set target (winners).
    features = team_data[feature_columns]
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i,1:27]["team"])

In [10]:
predict_the_winner(model_100, 2018, team_data)


WS Probability = 0.2
Philadelphia Phillies

WS Probability = 0.17
Pittsburgh Pirates

WS Probability = 0.17
Atlanta Braves

WS Probability = 0.17
Los Angeles Angels

WS Probability = 0.16
Texas Rangers

WS Probability = 0.15
Boston Red Sox

WS Probability = 0.14
Baltimore Orioles

WS Probability = 0.14
Cleveland Indians

WS Probability = 0.14
Washington Nationals

WS Probability = 0.13
New York Mets

WS Probability = 0.13
Arizona Diamondbacks

WS Probability = 0.12
Colorado Rockies

WS Probability = 0.12
Seattle Mariners

WS Probability = 0.12
Tampa Bay Rays

WS Probability = 0.12
Milwaukee Brewers

WS Probability = 0.11
Minnesota Twins

WS Probability = 0.11
Chicago White Sox

WS Probability = 0.11
Kansas City Royals

WS Probability = 0.11
Houston Astros

WS Probability = 0.1
Toronto Blue Jays

WS Probability = 0.1
Oakland Athletics

WS Probability = 0.1
San Francisco Giants

WS Probability = 0.09
Los Angeles Dodgers

WS Probability = 0.09
New York Yankees

WS Probability = 0.08
Detr

In [11]:
predict_the_winner(model_50, 2018, team_data)


WS Probability = 0.23
Texas Rangers

WS Probability = 0.2
Los Angeles Angels

WS Probability = 0.2
Milwaukee Brewers

WS Probability = 0.19
Philadelphia Phillies

WS Probability = 0.18
Atlanta Braves

WS Probability = 0.17
Houston Astros

WS Probability = 0.17
Boston Red Sox

WS Probability = 0.16
Pittsburgh Pirates

WS Probability = 0.16
Seattle Mariners

WS Probability = 0.16
Oakland Athletics

WS Probability = 0.16
Kansas City Royals

WS Probability = 0.15
Chicago Cubs

WS Probability = 0.14
Chicago White Sox

WS Probability = 0.14
New York Mets

WS Probability = 0.14
Tampa Bay Rays

WS Probability = 0.14
Cleveland Indians

WS Probability = 0.14
Arizona Diamondbacks

WS Probability = 0.14
Toronto Blue Jays

WS Probability = 0.13
Colorado Rockies

WS Probability = 0.13
Minnesota Twins

WS Probability = 0.13
St. Louis Cardinals

WS Probability = 0.12
Washington Nationals

WS Probability = 0.12
Detroit Tigers

WS Probability = 0.12
New York Yankees

WS Probability = 0.12
Cincinnati Re

In [12]:
predict_the_winner(model_25, 2018, team_data)


WS Probability = 0.28
Pittsburgh Pirates

WS Probability = 0.28
Texas Rangers

WS Probability = 0.26
Atlanta Braves

WS Probability = 0.25
Philadelphia Phillies

WS Probability = 0.24
Los Angeles Angels

WS Probability = 0.23
Houston Astros

WS Probability = 0.22
Boston Red Sox

WS Probability = 0.22
Seattle Mariners

WS Probability = 0.21
Los Angeles Dodgers

WS Probability = 0.21
Milwaukee Brewers

WS Probability = 0.2
New York Yankees

WS Probability = 0.19
Minnesota Twins

WS Probability = 0.19
Baltimore Orioles

WS Probability = 0.18
Kansas City Royals

WS Probability = 0.18
Arizona Diamondbacks

WS Probability = 0.17
Detroit Tigers

WS Probability = 0.17
Cleveland Indians

WS Probability = 0.17
Toronto Blue Jays

WS Probability = 0.16
Washington Nationals

WS Probability = 0.16
New York Mets

WS Probability = 0.15
Chicago White Sox

WS Probability = 0.15
Colorado Rockies

WS Probability = 0.15
Cincinnati Reds

WS Probability = 0.14
Tampa Bay Rays

WS Probability = 0.14
Oakland A

In [13]:
predict_the_winner(model_12, 2018, team_data)


WS Probability = 0.38
Pittsburgh Pirates

WS Probability = 0.38
Atlanta Braves

WS Probability = 0.37
Philadelphia Phillies

WS Probability = 0.37
Minnesota Twins

WS Probability = 0.35
Texas Rangers

WS Probability = 0.34
Los Angeles Angels

WS Probability = 0.32
Arizona Diamondbacks

WS Probability = 0.32
Seattle Mariners

WS Probability = 0.32
Oakland Athletics

WS Probability = 0.31
Boston Red Sox

WS Probability = 0.3
New York Mets

WS Probability = 0.3
Toronto Blue Jays

WS Probability = 0.3
Chicago White Sox

WS Probability = 0.3
Washington Nationals

WS Probability = 0.28
San Francisco Giants

WS Probability = 0.28
Milwaukee Brewers

WS Probability = 0.27
New York Yankees

WS Probability = 0.26
San Diego Padres

WS Probability = 0.25
Tampa Bay Rays

WS Probability = 0.25
Detroit Tigers

WS Probability = 0.24
Miami Marlins

WS Probability = 0.24
Baltimore Orioles

WS Probability = 0.24
Kansas City Royals

WS Probability = 0.24
Los Angeles Dodgers

WS Probability = 0.23
Cincinna

In [14]:
predict_the_winner(model_6, 2018, team_data)


WS Probability = 0.34
Arizona Diamondbacks

WS Probability = 0.33
Los Angeles Angels

WS Probability = 0.3
Washington Nationals

WS Probability = 0.29
Philadelphia Phillies

WS Probability = 0.29
Texas Rangers

WS Probability = 0.29
San Francisco Giants

WS Probability = 0.28
Minnesota Twins

WS Probability = 0.28
Boston Red Sox

WS Probability = 0.28
Milwaukee Brewers

WS Probability = 0.27
Seattle Mariners

WS Probability = 0.26
Cleveland Indians

WS Probability = 0.26
Atlanta Braves

WS Probability = 0.26
New York Yankees

WS Probability = 0.26
St. Louis Cardinals

WS Probability = 0.25
Miami Marlins

WS Probability = 0.25
Oakland Athletics

WS Probability = 0.25
Pittsburgh Pirates

WS Probability = 0.24
Toronto Blue Jays

WS Probability = 0.23
Houston Astros

WS Probability = 0.23
Chicago White Sox

WS Probability = 0.22
Tampa Bay Rays

WS Probability = 0.22
Kansas City Royals

WS Probability = 0.22
New York Mets

WS Probability = 0.22
Baltimore Orioles

WS Probability = 0.21
Cinc

In [15]:
predict_the_winner(model_3, 2018, team_data)


WS Probability = 0.28
Houston Astros

WS Probability = 0.27
New York Yankees

WS Probability = 0.24
Arizona Diamondbacks

WS Probability = 0.23
Cleveland Indians

WS Probability = 0.22
Los Angeles Dodgers

WS Probability = 0.22
Minnesota Twins

WS Probability = 0.22
Boston Red Sox

WS Probability = 0.2
Philadelphia Phillies

WS Probability = 0.2
Washington Nationals

WS Probability = 0.19
Los Angeles Angels

WS Probability = 0.17
Toronto Blue Jays

WS Probability = 0.16
Kansas City Royals

WS Probability = 0.16
Tampa Bay Rays

WS Probability = 0.16
Detroit Tigers

WS Probability = 0.15
Baltimore Orioles

WS Probability = 0.15
Seattle Mariners

WS Probability = 0.15
Atlanta Braves

WS Probability = 0.15
Oakland Athletics

WS Probability = 0.14
Chicago Cubs

WS Probability = 0.14
Miami Marlins

WS Probability = 0.13
Colorado Rockies

WS Probability = 0.13
San Francisco Giants

WS Probability = 0.13
Pittsburgh Pirates

WS Probability = 0.13
Cincinnati Reds

WS Probability = 0.13
Milwauke

In [16]:
predict_the_winner(model_100, 2017, team_data)


WS Probability = 0.12
Seattle Mariners

WS Probability = 0.12
Washington Nationals

WS Probability = 0.12
New York Mets

WS Probability = 0.1
Arizona Diamondbacks

WS Probability = 0.1
Los Angeles Angels

WS Probability = 0.1
Minnesota Twins

WS Probability = 0.09
Pittsburgh Pirates

WS Probability = 0.08
San Francisco Giants

WS Probability = 0.08
Los Angeles Dodgers

WS Probability = 0.08
Cleveland Indians

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.07
Toronto Blue Jays

WS Probability = 0.07
Atlanta Braves

WS Probability = 0.06
Tampa Bay Rays

WS Probability = 0.06
Chicago White Sox

WS Probability = 0.05
Boston Red Sox

WS Probability = 0.05
Milwaukee Brewers

WS Probability = 0.05
Philadelphia Phillies

WS Probability = 0.05
Texas Rangers

WS Probability = 0.04
Miami Marlins

WS Probability = 0.04
New York Yankees

WS Probability = 0.04
Colorado Rockies

WS Probability = 0.03
Baltimore Orioles

WS Probability = 0.03
St. Louis Cardinals

WS Probability = 0.03
Oaklan

In [17]:
predict_the_winner(model_50, 2017, team_data)


WS Probability = 0.17
Minnesota Twins

WS Probability = 0.17
Detroit Tigers

WS Probability = 0.16
Cleveland Indians

WS Probability = 0.15
Los Angeles Angels

WS Probability = 0.15
Toronto Blue Jays

WS Probability = 0.13
Washington Nationals

WS Probability = 0.11
Arizona Diamondbacks

WS Probability = 0.11
Pittsburgh Pirates

WS Probability = 0.11
New York Yankees

WS Probability = 0.11
Seattle Mariners

WS Probability = 0.09
San Francisco Giants

WS Probability = 0.09
New York Mets

WS Probability = 0.08
Baltimore Orioles

WS Probability = 0.08
St. Louis Cardinals

WS Probability = 0.07
Los Angeles Dodgers

WS Probability = 0.06
Tampa Bay Rays

WS Probability = 0.06
Miami Marlins

WS Probability = 0.06
Atlanta Braves

WS Probability = 0.06
Boston Red Sox

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.05
Chicago Cubs

WS Probability = 0.04
San Diego Padres

WS Probability = 0.04
Kansas City Royals

WS Probability = 0.04
Chicago White Sox

WS Probability = 0.03
Oak

In [18]:
predict_the_winner(model_25, 2017, team_data)


WS Probability = 0.2
New York Yankees

WS Probability = 0.2
Los Angeles Dodgers

WS Probability = 0.17
Cleveland Indians

WS Probability = 0.17
Pittsburgh Pirates

WS Probability = 0.15
Boston Red Sox

WS Probability = 0.13
Tampa Bay Rays

WS Probability = 0.13
Houston Astros

WS Probability = 0.13
Milwaukee Brewers

WS Probability = 0.12
Oakland Athletics

WS Probability = 0.12
Washington Nationals

WS Probability = 0.1
Seattle Mariners

WS Probability = 0.08
Atlanta Braves

WS Probability = 0.08
Chicago Cubs

WS Probability = 0.08
Los Angeles Angels

WS Probability = 0.08
Detroit Tigers

WS Probability = 0.07
Toronto Blue Jays

WS Probability = 0.07
New York Mets

WS Probability = 0.07
Philadelphia Phillies

WS Probability = 0.06
St. Louis Cardinals

WS Probability = 0.06
Minnesota Twins

WS Probability = 0.05
Baltimore Orioles

WS Probability = 0.05
Arizona Diamondbacks

WS Probability = 0.03
Miami Marlins

WS Probability = 0.03
Chicago White Sox

WS Probability = 0.03
San Francisc

In [19]:
predict_the_winner(model_12, 2017, team_data)


WS Probability = 0.28
Washington Nationals

WS Probability = 0.28
Los Angeles Dodgers

WS Probability = 0.27
New York Yankees

WS Probability = 0.21
Boston Red Sox

WS Probability = 0.2
Houston Astros

WS Probability = 0.18
Los Angeles Angels

WS Probability = 0.16
Tampa Bay Rays

WS Probability = 0.15
Cleveland Indians

WS Probability = 0.14
Detroit Tigers

WS Probability = 0.13
Oakland Athletics

WS Probability = 0.12
Minnesota Twins

WS Probability = 0.12
Seattle Mariners

WS Probability = 0.11
Milwaukee Brewers

WS Probability = 0.11
Pittsburgh Pirates

WS Probability = 0.11
St. Louis Cardinals

WS Probability = 0.11
New York Mets

WS Probability = 0.11
Atlanta Braves

WS Probability = 0.08
Miami Marlins

WS Probability = 0.08
Colorado Rockies

WS Probability = 0.08
Arizona Diamondbacks

WS Probability = 0.07
Chicago Cubs

WS Probability = 0.07
Toronto Blue Jays

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.05
Chicago White Sox

WS Probability = 0.05
Texas Range

In [20]:
predict_the_winner(model_6, 2017, team_data)


WS Probability = 0.23
Houston Astros

WS Probability = 0.23
New York Yankees

WS Probability = 0.23
Washington Nationals

WS Probability = 0.17
Oakland Athletics

WS Probability = 0.16
Cleveland Indians

WS Probability = 0.15
Boston Red Sox

WS Probability = 0.14
Los Angeles Dodgers

WS Probability = 0.12
Baltimore Orioles

WS Probability = 0.12
Detroit Tigers

WS Probability = 0.11
Los Angeles Angels

WS Probability = 0.1
Toronto Blue Jays

WS Probability = 0.08
Pittsburgh Pirates

WS Probability = 0.07
Atlanta Braves

WS Probability = 0.07
Tampa Bay Rays

WS Probability = 0.07
New York Mets

WS Probability = 0.06
Seattle Mariners

WS Probability = 0.06
Milwaukee Brewers

WS Probability = 0.05
Minnesota Twins

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.05
Kansas City Royals

WS Probability = 0.04
Chicago Cubs

WS Probability = 0.04
Chicago White Sox

WS Probability = 0.04
Philadelphia Phillies

WS Probability = 0.03
Miami Marlins

WS Probability = 0.03
Arizona Dia

In [21]:
predict_the_winner(model_3, 2017, team_data)


WS Probability = 0.24
Houston Astros

WS Probability = 0.24
Boston Red Sox

WS Probability = 0.23
Washington Nationals

WS Probability = 0.22
New York Yankees

WS Probability = 0.17
Los Angeles Dodgers

WS Probability = 0.17
Oakland Athletics

WS Probability = 0.14
Tampa Bay Rays

WS Probability = 0.14
Cleveland Indians

WS Probability = 0.13
Detroit Tigers

WS Probability = 0.12
Toronto Blue Jays

WS Probability = 0.11
St. Louis Cardinals

WS Probability = 0.1
Arizona Diamondbacks

WS Probability = 0.1
Los Angeles Angels

WS Probability = 0.1
Baltimore Orioles

WS Probability = 0.09
Atlanta Braves

WS Probability = 0.09
Pittsburgh Pirates

WS Probability = 0.08
New York Mets

WS Probability = 0.08
Minnesota Twins

WS Probability = 0.08
Seattle Mariners

WS Probability = 0.07
Chicago Cubs

WS Probability = 0.06
Milwaukee Brewers

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.04
Colorado Rockies

WS Probability = 0.04
Philadelphia Phillies

WS Probability = 0.03
San Di

In [22]:
predict_the_winner(model_100, 2016, team_data)


WS Probability = 0.14
Washington Nationals

WS Probability = 0.14
Kansas City Royals

WS Probability = 0.13
Los Angeles Dodgers

WS Probability = 0.12
Chicago Cubs

WS Probability = 0.11
Texas Rangers

WS Probability = 0.1
Milwaukee Brewers

WS Probability = 0.1
Los Angeles Angels

WS Probability = 0.1
Boston Red Sox

WS Probability = 0.09
St. Louis Cardinals

WS Probability = 0.09
Colorado Rockies

WS Probability = 0.08
Seattle Mariners

WS Probability = 0.08
New York Yankees

WS Probability = 0.07
Minnesota Twins

WS Probability = 0.06
Arizona Diamondbacks

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.06
Houston Astros

WS Probability = 0.05
Tampa Bay Rays

WS Probability = 0.05
Toronto Blue Jays

WS Probability = 0.04
San Diego Padres

WS Probability = 0.04
Miami Marlins

WS Probability = 0.04
Oakland Athletics

WS Probability = 0.04
San Francisco Giants

WS Probability = 0.04
Cleveland Indians

WS Probability = 0.03
Pittsburgh Pirates

WS Probability = 0.03
Cinc

In [23]:
predict_the_winner(model_50, 2016, team_data)


WS Probability = 0.22
Houston Astros

WS Probability = 0.18
Kansas City Royals

WS Probability = 0.13
Los Angeles Angels

WS Probability = 0.13
New York Yankees

WS Probability = 0.12
Washington Nationals

WS Probability = 0.12
Seattle Mariners

WS Probability = 0.11
Cleveland Indians

WS Probability = 0.1
Baltimore Orioles

WS Probability = 0.1
Chicago Cubs

WS Probability = 0.1
Oakland Athletics

WS Probability = 0.09
Toronto Blue Jays

WS Probability = 0.09
Los Angeles Dodgers

WS Probability = 0.09
Boston Red Sox

WS Probability = 0.09
Colorado Rockies

WS Probability = 0.09
Minnesota Twins

WS Probability = 0.08
New York Mets

WS Probability = 0.08
Tampa Bay Rays

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.07
Texas Rangers

WS Probability = 0.06
Cincinnati Reds

WS Probability = 0.06
St. Louis Cardinals

WS Probability = 0.05
Pittsburgh Pirates

WS Probability = 0.05
San Francisco Giants

WS Probability = 0.05
San Diego Padres

WS Probability = 0.05
Miami Marlins

W

In [24]:
predict_the_winner(model_25, 2016, team_data)


WS Probability = 0.26
Los Angeles Dodgers

WS Probability = 0.25
Houston Astros

WS Probability = 0.23
Boston Red Sox

WS Probability = 0.18
Washington Nationals

WS Probability = 0.15
Seattle Mariners

WS Probability = 0.15
New York Yankees

WS Probability = 0.13
Baltimore Orioles

WS Probability = 0.13
Chicago Cubs

WS Probability = 0.12
Cleveland Indians

WS Probability = 0.12
Los Angeles Angels

WS Probability = 0.1
Minnesota Twins

WS Probability = 0.08
Tampa Bay Rays

WS Probability = 0.07
San Diego Padres

WS Probability = 0.07
Oakland Athletics

WS Probability = 0.07
Kansas City Royals

WS Probability = 0.06
Arizona Diamondbacks

WS Probability = 0.06
Texas Rangers

WS Probability = 0.06
Miami Marlins

WS Probability = 0.05
St. Louis Cardinals

WS Probability = 0.04
Colorado Rockies

WS Probability = 0.04
Toronto Blue Jays

WS Probability = 0.04
Detroit Tigers

WS Probability = 0.04
Cincinnati Reds

WS Probability = 0.03
New York Mets

WS Probability = 0.02
Pittsburgh Pirates


In [25]:
predict_the_winner(model_12, 2016, team_data)


WS Probability = 0.26
Washington Nationals

WS Probability = 0.24
Los Angeles Dodgers

WS Probability = 0.23
Houston Astros

WS Probability = 0.22
Cleveland Indians

WS Probability = 0.22
New York Yankees

WS Probability = 0.2
Los Angeles Angels

WS Probability = 0.16
Chicago Cubs

WS Probability = 0.15
Seattle Mariners

WS Probability = 0.14
Baltimore Orioles

WS Probability = 0.1
Toronto Blue Jays

WS Probability = 0.08
Minnesota Twins

WS Probability = 0.08
Tampa Bay Rays

WS Probability = 0.07
Detroit Tigers

WS Probability = 0.06
Colorado Rockies

WS Probability = 0.06
Miami Marlins

WS Probability = 0.06
St. Louis Cardinals

WS Probability = 0.06
Philadelphia Phillies

WS Probability = 0.05
Arizona Diamondbacks

WS Probability = 0.05
Milwaukee Brewers

WS Probability = 0.05
Oakland Athletics

WS Probability = 0.04
Kansas City Royals

WS Probability = 0.04
Texas Rangers

WS Probability = 0.04
Boston Red Sox

WS Probability = 0.03
San Francisco Giants

WS Probability = 0.02
Pittsb

In [26]:
predict_the_winner(model_6, 2016, team_data)


WS Probability = 0.21
Los Angeles Dodgers

WS Probability = 0.19
Cleveland Indians

WS Probability = 0.18
New York Yankees

WS Probability = 0.18
Houston Astros

WS Probability = 0.16
Washington Nationals

WS Probability = 0.14
Los Angeles Angels

WS Probability = 0.13
Seattle Mariners

WS Probability = 0.1
Minnesota Twins

WS Probability = 0.1
Tampa Bay Rays

WS Probability = 0.09
Chicago Cubs

WS Probability = 0.08
Colorado Rockies

WS Probability = 0.08
Baltimore Orioles

WS Probability = 0.08
Boston Red Sox

WS Probability = 0.07
Kansas City Royals

WS Probability = 0.07
New York Mets

WS Probability = 0.06
San Francisco Giants

WS Probability = 0.06
Arizona Diamondbacks

WS Probability = 0.06
Detroit Tigers

WS Probability = 0.05
Oakland Athletics

WS Probability = 0.04
Toronto Blue Jays

WS Probability = 0.04
Milwaukee Brewers

WS Probability = 0.03
St. Louis Cardinals

WS Probability = 0.03
Cincinnati Reds

WS Probability = 0.02
Philadelphia Phillies

WS Probability = 0.02
Miam

In [27]:
predict_the_winner(model_3, 2016, team_data)


WS Probability = 0.37
Houston Astros

WS Probability = 0.3
New York Yankees

WS Probability = 0.21
Boston Red Sox

WS Probability = 0.21
Los Angeles Dodgers

WS Probability = 0.16
Cleveland Indians

WS Probability = 0.13
Seattle Mariners

WS Probability = 0.12
Washington Nationals

WS Probability = 0.11
Tampa Bay Rays

WS Probability = 0.1
Chicago Cubs

WS Probability = 0.1
Detroit Tigers

WS Probability = 0.09
Minnesota Twins

WS Probability = 0.08
Miami Marlins

WS Probability = 0.08
Los Angeles Angels

WS Probability = 0.07
Arizona Diamondbacks

WS Probability = 0.07
Oakland Athletics

WS Probability = 0.06
Texas Rangers

WS Probability = 0.06
Kansas City Royals

WS Probability = 0.06
Baltimore Orioles

WS Probability = 0.05
Pittsburgh Pirates

WS Probability = 0.05
Toronto Blue Jays

WS Probability = 0.05
Philadelphia Phillies

WS Probability = 0.05
Milwaukee Brewers

WS Probability = 0.04
New York Mets

WS Probability = 0.04
San Diego Padres

WS Probability = 0.03
St. Louis Cardi

not sure.  save it.